In [218]:
# !pip install requests_html
# !pip install PyPDF2
# !pip install rouge
# !pip install openpyxl


In [219]:
from requests_html import HTMLSession
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
import re
from openpyxl import Workbook

In [220]:
s= HTMLSession()

In [221]:
def clean_string(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = text.replace('\n', '')
    return text

def convert_to_map(lst):
    map_dict = {item.split(':')[0].strip(): item.split(':')[1].strip() for item in lst}
    return map_dict

In [222]:
headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.57'}
file= open('input.txt','r')
ids= file.readlines()
base_url='https://pubmed.ncbi.nlm.nih.gov/'
total_abstracts={}
for id in ids:
    pmcid= id.strip()
    r=s.get(base_url+pmcid+'/',headers=headers,timeout=5)
    soup = BeautifulSoup(r.content, 'html.parser')
    div_abstract = soup.find('div', {'class': 'abstract'})

    if div_abstract:
        abstract_components= div_abstract.find_all('p')
        abstract_components=[clean_string(abstract.text.strip()) for abstract in abstract_components]
        total_abstracts[id]=convert_to_map(abstract_components)

In [224]:
def write_nested_map_to_excel(nested_map, filename):
    workbook = Workbook()
    sheet = workbook.active
    row = 1
    for outer_key, inner_map in nested_map.items():
        sheet.cell(row=row, column=1, value=outer_key)
        for inner_key, value in inner_map.items():
            sheet.cell(row=row, column=2, value=inner_key)
            sheet.cell(row=row, column=3, value=value)
            row += 1
        row += 1
    workbook.save(filename)

In [225]:
write_nested_map_to_excel(total_abstracts, 'output.xlsx')